## Decorators

Decorators were added in py to make func and method wrapping easier to read and understand. The original use case was to be able to define the methods as class methods or static methods.

Anything that is **callable** (any object that implements the __call__ method is considered callable), can be used as as decorator.

The decorator syntax is simply only a syntactic sugar.

```python
@some_decorator
def decorated_f():
    pass
    
# can be replaced by

def decorated_f():
    pass
    
decorated_f = some_decorator(decorated_f)
```

In [1]:
# as a func
def mydecorator(func):
    def wrapped(*args, **kwargs):
        print('do something before the original func call')
        result = func(*args, **kwargs)
        print('do something after the original func call')
        return result
    
    return wrapped


@mydecorator
def add(a, b):
    return a + b


add(1, 2)

do something before the original func call
do something after the original func call


3

In [3]:
# as a class
class MyDecorator:
    def __init__(self, func):
        self.func = func
        
    def __call__(self, *args, **kwargs):
        print('do something before the original func call')
        result = self.func(*args, **kwargs)
        print('do something after the original func call')
        return result
    

@MyDecorator
def sub(a, b):
    return a - b

sub(1, 2)

do something before the original func call
do something after the original func call


-1

### Parameterizing decorators

In [6]:
def repeat(n=3):
    def actual_decorator(func):
        def wrapped(*args, **kwargs):
            result = None
            for _ in range(n):
                result = func(*args, **kwargs)
            return result
        return wrapped
    
    return actual_decorator


@repeat(n=2)
def foo():
    print('foo')
    

foo()

foo
foo


### Preseving decorators

Common pitfalls of using decorators is not preserving function metadata (mostly docstring and original name) when using decorators.

A proper solution is to use the `wraps()` decorator.

## Usages

The common patterns for decorators are:

* Argument checking
* Caching
* Proxy
* Context provider


In [13]:
# caching

import time
import hashlib
import pickle

cache = {}


def is_obsolete(entry, duration):
    return time.time() - entry['time'] > duration


def compute_key(func, args, kwargs):
    key = pickle.dumps((func.__name__, args, kwargs))
    return hashlib.sha1(key).hexdigest()

def memoize(duration=10):
    def _memoize(func):
        def __memoize(*args, **kwargs):
            key = compute_key(func, args, kwargs)
            if (key in cache and not is_obsolete(cache[key], duration)):
                print('got from caching:)')
                return cache[key]['value']
            
            print('compute value...')
            result = func(*args, **kwargs)
            cache[key] = {'value': result, 'time': time.time()}
            return result
        return __memoize
    return _memoize

In [14]:
@memoize(3)
def complex_sutff(a, b):
    return a + b

In [22]:
complex_sutff(1, 2)
complex_sutff(1, 2)
time.sleep(3.1)
complex_sutff(1, 2)

compute value...
got from caching:)
compute value...


3

In [23]:
cache

{'a5e3c203a67a9e2a4ddf12bda34fcc9c934fcb6d': {'value': 3,
  'time': 1546708331.893444}}